## LangChain Expression Language Based Memory
- memory에서 기억하고 있는 내용을 custom chain에 주입하는 방법입니다.
- Custom chain을 생성해야 할 상황이 많이 있습니다.
- 따라서, 해당 기법은 memory를 적재적소에 사용하기 위하여 많은 도움이 될 것이라고 생각합니다!

In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [3]:
llm = ChatOpenAI(temperature=0.2)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    return_messages=True,
    max_token_limit=100,
    memory_key="chat_history"
)

prompt = ChatPromptTemplate.from_messages(
    messages=[
        ("system", "You are a helpful AI talking to human!"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}")
    ]
)

In [4]:
# custom chain에서 memory에 저장된 컨텐츠를 불러오는 함수입니다. 
def load_history(input):
    return memory.load_memory_variables({})["chat_history"]

In [5]:
# chain을 생성합니다.
chain = RunnablePassthrough.assign(chat_history=load_history) | prompt | llm

In [6]:
# chain을 실행할 함수를 정의합니다.
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        inputs={"input": question},
        outputs={"output": result.content}
    )
    print(result)

In [7]:
invoke_chain("My name is jiho!")

content='Hello Jiho! How can I assist you today?'


In [8]:
invoke_chain("Whay is my name?")

content='Your name is Jiho.'
